In [1]:
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import glob
import os
import matplotlib.cm as cm


In [5]:
# ----------------------------------------------------------------------------
# Paths
# ----------------------------------------------------------------------------
path_data="./paleo_sea_level/sea_level_data/Greenland/*"
path_output="./output"
data_paths = sorted(glob.glob(path_data))
grid = xr.open_dataset("/p/projects/megarun/ice_data/Greenland/GRL-8KM/GRL-8KM_TOPO-M17.nc")
ds_new=xr.open_dataset("./output/rsl_dataset.nc")
ds_region=xr.open_dataset("./output/rsl_dataset_reduced.nc")

In [8]:
def define_labels(reg,x,y):
    if reg == "Tuttulissuaq":
        return -300, -20
    elif reg == "Thule":
        return -150, -50
    elif reg == "Nordvestoe":
        return -100, 50
    elif reg == "Hall_Land":
        return -350, 50
    elif reg == "Germania_Land":
        return 50, 50
    elif reg == "Bessel_Fjord":
        return -350, 0
    elif reg == "Kangerluarsuk_Cass_Fjord":
        return -350, -100
    elif reg == "Warming_Land":
        return -50, -90
    elif reg == "Nansen_land":  
        return -350, -10
    elif reg == "Kap_Morris_Jesup":
        return -300, 50  
    elif reg == "JP_Koch_Fjord":
        return -60, -80
    elif reg == "Nioghalvfjerdsfjorden":  
        return 100, -30
    elif reg == "Prinsesse_Ingeborg_Halvoe":
        return 120, -60
    elif reg == "Kap_Clarence_Wyckoff":
        return 30, 50
    elif reg == "Frederick_E_Hyde_Fjord":
        return -550, 80
    elif reg == "Independence_Fjord":
        return 200, 20
    elif reg == "Renland":
        return -250, -50
    elif reg == "Traill_Oe":
        return 100, 50
    elif reg == "Kempe_Fjord":    
        return -350, -50
    elif reg == "Kong_Oscar_Fjord":    
        return 180, 0
    elif reg == "Schuchert_Dal":    
        return 90, -30
    elif reg == "Jameson_Land":
        return 100, -50
    elif reg == "Kapisillit":    
        return 60, 0
    elif reg == "Nuuk":    
        return -150, 30
    elif reg == "Akulliit":    
        return 50, -20
    elif reg == "Tasiussarsuaq":   
        return -350, 0
    elif reg == "Qeqertarsuatsiaat":   
        return -400, -50
    elif reg == "Kangerlussuaq":    
        return 50, 50
    elif reg == "Alluttoq_Island":    
        return 50, 0
    elif reg == "Young_sund":    
        return 80, 2
    elif reg == "Hold_With_Hope":    
        return 250, 0  
    elif reg == "Ikertooq_Fjord":    
        return 40, -80
    elif reg == "Itilleq":
        return -150, -20
    elif reg == "Ilulissat":
        return 50, 0   
    elif reg == "Danmarks_Fjord":
        return 150, -50   
    elif reg == "Inglefield_Fjord":
        return 50, 0   
    else:
        if x>0:
            return 50, 0
        else:
            return -250, 0
    

In [ ]:
present1=xr.open_dataset("/p/projects/megarun/ice_data/Greenland/GRL-4KM/GRL-4KM_TOPO-M17.nc")
present=present1.where((present1.xc<900)&(present1.yc<present1.xc*0.75-650))

ds_all = ds_new

f=17
fig,ax=plt.subplots(1,1, figsize=(12,15))
ax.set_facecolor("white")
ax.contour(present.xc, present.yc, present.z_srf, levels=np.linspace(0,10,1), colors="black", linewidths=0.5,zorder=3)
ax.contourf(present.xc, present.yc, present.z_srf.where(present.z_srf>-10), colors="#98C37C",zorder=1)

xmin=-800
xmax=1300
mask = np.where(np.isnan(present.z_srf), 0, present.z_srf)
mask = np.where(mask == 0, 1, np.nan)
ny = mask.shape[0]
xc = present.xc.values
zeros_left = np.ones((ny, 1))
zeros_right = np.ones((ny, 1))
mask_extended = np.hstack([zeros_left, mask, zeros_right])
xc_extended = np.concatenate([[xmin], xc, [xmax]])

ax.contourf(xc_extended, present.yc, mask_extended,colors="white",zorder=2)

ax.contour(present.xc, present.yc, present.H_ice, levels=np.linspace(0,10,1), colors="black", linewidths=0.5,zorder=3)
ax.contourf(present.xc, present.yc, present.H_ice.where(present.H_ice>0), levels=np.linspace(0,4000,2), colors="#E6F1F3",zorder=3)

prefix_to_color = {}
cmap = cm.get_cmap('tab20')  # paleta con muchos colores
color_index = 0
regions = np.unique(ds.region.values)
colors = plt.cm.tab20(np.linspace(0, 1, len(regions)))
# alternativas: tab20, Set2, viridis, etc.

    
    
for r_idx, reg in enumerate(regions):
    color = colors[r_idx]   # color único para esta región
    sim = ds_new.where(ds_new.region == reg, drop=True)

    i = 0
    for site in sim.site.values:
        sim1 = ds_all.sel(site=site)
        x, y = sim1.xc.values, sim1.yc.values

        ax.plot(
            x, y, 'o',
            markersize=10,
            markeredgecolor='black',
            markerfacecolor=color,
            label=str(reg) if i == 0 else None,  # una entrada por región
            zorder=4
        )

        a, b = define_labels(reg,x,y)
        if i == 0:
            ax.text(x+a, y+b, str(reg), color="black", fontsize=f, zorder=5,
            bbox=dict(facecolor=color,edgecolor='none',alpha=0.5))

        i += 1

    

ax.set_xlim(xmin,xmax)
ax.set_aspect(1)
# ax.tick_params(labelbottom=False, labelleft=False)
plt.tight_layout()
fig.savefig("./output/map_RSL.png", dpi=300)
plt.close()

/tmp/ipykernel_3765187/3908914943.py:29: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20')  # paleta con muchos colores


In [11]:
present1=xr.open_dataset("/p/projects/megarun/ice_data/Greenland/GRL-4KM/GRL-4KM_TOPO-M17.nc")
present=present1.where((present1.xc<900)&(present1.yc<present1.xc*0.75-650))

ds_all = ds_region

f=17
fig,ax=plt.subplots(1,1, figsize=(12,15))
ax.set_facecolor("white")
ax.contour(present.xc, present.yc, present.z_srf, levels=np.linspace(0,10,1), colors="black", linewidths=0.5,zorder=3)
ax.contourf(present.xc, present.yc, present.z_srf.where(present.z_srf>-10), colors="#98C37C",zorder=1)

xmin=-800
xmax=1300
mask = np.where(np.isnan(present.z_srf), 0, present.z_srf)
mask = np.where(mask == 0, 1, np.nan)
ny = mask.shape[0]
xc = present.xc.values
zeros_left = np.ones((ny, 1))
zeros_right = np.ones((ny, 1))
mask_extended = np.hstack([zeros_left, mask, zeros_right])
xc_extended = np.concatenate([[xmin], xc, [xmax]])

ax.contourf(xc_extended, present.yc, mask_extended,colors="white",zorder=2)

ax.contour(present.xc, present.yc, present.H_ice, levels=np.linspace(0,10,1), colors="black", linewidths=0.5,zorder=3)
ax.contourf(present.xc, present.yc, present.H_ice.where(present.H_ice>0), levels=np.linspace(0,4000,2), colors="#E6F1F3",zorder=3)

prefix_to_color = {}
cmap = cm.get_cmap('tab20')  # paleta con muchos colores
color_index = 0
regions = np.unique(ds_all.region.values)
colors = plt.cm.tab20(np.linspace(0, 1, len(regions)))

for r_idx, reg in enumerate(regions):
    color = colors[r_idx]   # color único para esta región
    sim = ds_all.sel(region = reg)

    x, y = sim.xc.values, sim.yc.values

    ax.plot(
        x, y, 'o',
        markersize=15,
        markeredgecolor='black',
        markerfacecolor=color,
        label=str(reg),  # una entrada por región
        zorder=4
    )

    a, b = define_labels(reg,x,y)

    ax.text(x+a, y+b, str(reg), color="black", fontsize=f, zorder=5,
    bbox=dict(facecolor=color,edgecolor='none',alpha=0.5))

    

ax.set_xlim(xmin,xmax)
ax.set_aspect(1)
# ax.tick_params(labelbottom=False, labelleft=False)
plt.tight_layout()
fig.savefig("./output/map_RSL_reduced.png", dpi=300)
plt.close()

/tmp/ipykernel_832154/3699980536.py:29: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('tab20')  # paleta con muchos colores
